In [2]:
!pip install tqdm>=4.62.2
!pip install Levenshtein
!pip install sentence-transformers

zsh:1: 4.62.2 not found


In [3]:
import pandas as pd
from Levenshtein import distance
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np


/tmp/ipykernel_462737/2832460321.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/philipp/.pyenv/versions/3.10.6/envs/movie_recommendation_GPT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('../raw_data/mpst_full_data.csv')

In [5]:
df['plot_word_count'] = df['plot_synopsis'].apply(lambda x: len(x.split()))
df

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,plot_word_count
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb,1315
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb,378
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb,707
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb,2317
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb,3063
...,...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",test,wikipedia,441
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",train,wikipedia,546
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,train,wikipedia,415
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,test,wikipedia,378


In [6]:
df_500 = df[df['plot_word_count'] < 500].copy()
df_500 = df_500.drop(columns = 'plot_word_count', axis = 1)

In [7]:
df

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,plot_word_count
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb,1315
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb,378
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb,707
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb,2317
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb,3063
...,...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",test,wikipedia,441
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",train,wikipedia,546
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,train,wikipedia,415
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,test,wikipedia,378


In [29]:
df_100 = df[df['plot_word_count'] < 120].copy()
df_100 = df_100.drop(columns = 'plot_word_count', axis = 1)
df_100

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
707,tt1758575,Blue Like Jazz,"""I never liked jazz music because jazz music d...",philosophical,val,imdb
4562,tt0088693,Alice in Wonderland,"""The incredible all-star version of the unforg...",violence,val,imdb
5907,tt0418325,Die Wilden Kerle 2,In the English translation of the Wild Soccer ...,fantasy,train,wikipedia
5946,tt4773278,Verdades Secretas,"Fame, power and money. Skin-deep pleasure. Pul...","revenge, murder",train,wikipedia
9523,tt1922373,Thor & Loki: Blood Brothers,"Loki has become the ruler of Asgard. However, ...",good versus evil,train,wikipedia
11702,tt0040571,Mark of the Lash,The movie broke into the following chapters:\n...,murder,train,wikipedia
12196,tt0096477,Yateem,Story revolves around a police officer who kil...,revenge,train,wikipedia


In [8]:
# model = SentenceTransformer('all-MiniLM-L6-v2')
# df_ = df_500.copy()
# df_['plot_synopsis'] = df_['plot_synopsis'].apply(lambda x : model.encode(x))
# df_index = df_.pop('title')
# df_ = df_[['plot_synopsis']]
# df_ = pd.DataFrame(np.column_stack(list(zip(*df_.values))))
# df_.index = df_index
# df_

In [9]:
model = SentenceTransformer('all-MiniLM-L6-v2')
df_ = df_500.copy()
df_['plot_synopsis'] = df_['plot_synopsis'].apply(lambda x : model.encode(x))


In [11]:
#df_save = df_.copy()
df_=df_save.copy()

In [12]:
df_index = df_.pop('title')
df_

,imdb_id,plot_synopsis,tags,split,synopsis_source
1,tt1733125,"[-0.05019064, 0.17130753, -0.011070515, 0.0311...",violence,train,imdb
8,tt0021079,"[0.0043492615, 0.016670505, -0.12862247, -0.03...",violence,train,imdb
12,tt0795493,"[-0.031534854, 0.06807867, -0.051520858, -0.02...","tragedy, dramatic, murder",train,wikipedia
19,tt0102007,"[-0.10105889, 0.015999699, 0.039534684, 0.1335...","paranormal, horror, haunting",test,imdb
45,tt4226388,"[-0.0070261783, 0.071364164, -0.076363906, 0.0...",tragedy,train,imdb
...,...,...,...,...,...
14820,tt0074646,"[-0.058952607, -0.032931328, -0.12741551, 0.04...",blaxploitation,train,wikipedia
14821,tt0025601,"[-0.059396524, -0.12155574, -0.01906538, -0.02...",romantic,val,wikipedia
14823,tt0219952,"[-0.07439944, 0.016631534, -0.004580092, -0.06...","comedy, murder",test,wikipedia
14825,tt0063443,"[-0.11095489, 0.063712165, -0.08971749, -0.025...",anti war,train,wikipedia


In [13]:
df_ = df_[['plot_synopsis']]
df_.columns


Index(['plot_synopsis'], dtype='object')

In [14]:
df_ = pd.DataFrame(np.column_stack(list(zip(*df_.values))))
df_

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.050191,0.171308,-0.011071,0.031128,-0.015905,-0.031009,0.032038,0.016664,0.024120,-0.005058,...,-0.054060,-0.116127,0.037026,0.015278,-0.036860,0.026114,0.054591,-0.077697,0.040902,0.020990
1,0.004349,0.016671,-0.128622,-0.034422,-0.017498,0.050455,0.079293,0.067014,-0.021096,0.005681,...,0.041640,-0.054807,-0.020852,0.032621,0.066752,-0.005349,-0.032227,0.039703,-0.048936,-0.108932
2,-0.031535,0.068079,-0.051521,-0.029258,-0.022146,0.011981,0.019802,-0.037174,0.071357,0.004537,...,-0.055937,-0.090939,-0.096302,0.070232,0.009827,0.042309,0.006600,-0.005772,0.020566,-0.081024
3,-0.101059,0.016000,0.039535,0.133585,-0.000026,-0.006421,-0.048751,-0.085075,0.102592,-0.079920,...,0.034891,-0.087435,0.104487,0.023701,-0.052540,0.021987,0.059501,-0.005483,-0.019163,0.026004
4,-0.007026,0.071364,-0.076364,0.005140,0.027067,0.127636,0.060012,0.040592,0.035122,-0.034440,...,0.040684,-0.063287,-0.074967,0.006059,-0.025546,0.050380,0.023971,-0.008307,-0.047381,-0.003047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4766,-0.058953,-0.032931,-0.127416,0.046263,-0.065068,-0.022222,0.100632,-0.101592,-0.031969,0.048770,...,-0.046479,-0.093698,0.006776,0.042431,0.090799,0.052930,-0.016827,0.006115,-0.118487,-0.011561
4767,-0.059397,-0.121556,-0.019065,-0.020089,-0.143418,0.031162,0.058256,0.016226,-0.019162,-0.144606,...,0.007160,-0.020355,0.011194,0.047112,0.085196,0.047241,-0.076121,-0.000444,0.044953,-0.091697
4768,-0.074399,0.016632,-0.004580,-0.062739,-0.085249,0.019829,0.132874,0.056214,0.041988,-0.068564,...,0.026873,-0.137031,-0.019027,-0.003781,-0.043318,-0.032821,-0.043339,-0.063091,-0.026474,0.057520
4769,-0.110955,0.063712,-0.089717,-0.025879,0.053711,0.042442,0.058297,-0.021060,-0.048976,0.052764,...,0.084501,-0.054149,-0.051150,-0.032155,-0.001647,0.067234,-0.008135,-0.051634,-0.060839,-0.024784


In [15]:
df_.index = df_index
df_

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
title,,,,,,,,,,,,,,,,,,,,,
Dungeons & Dragons: The Book of Vile Darkness,-0.050191,0.171308,-0.011071,0.031128,-0.015905,-0.031009,0.032038,0.016664,0.024120,-0.005058,...,-0.054060,-0.116127,0.037026,0.015278,-0.036860,0.026114,0.054591,-0.077697,0.040902,0.020990
Little Caesar,0.004349,0.016671,-0.128622,-0.034422,-0.017498,0.050455,0.079293,0.067014,-0.021096,0.005681,...,0.041640,-0.054807,-0.020852,0.032621,0.066752,-0.005349,-0.032227,0.039703,-0.048936,-0.108932
Cassandra's Dream,-0.031535,0.068079,-0.051521,-0.029258,-0.022146,0.011981,0.019802,-0.037174,0.071357,0.004537,...,-0.055937,-0.090939,-0.096302,0.070232,0.009827,0.042309,0.006600,-0.005772,0.020566,-0.081024
The Haunted,-0.101059,0.016000,0.039535,0.133585,-0.000026,-0.006421,-0.048751,-0.085075,0.102592,-0.079920,...,0.034891,-0.087435,0.104487,0.023701,-0.052540,0.021987,0.059501,-0.005483,-0.019163,0.026004
Victoria,-0.007026,0.071364,-0.076364,0.005140,0.027067,0.127636,0.060012,0.040592,0.035122,-0.034440,...,0.040684,-0.063287,-0.074967,0.006059,-0.025546,0.050380,0.023971,-0.008307,-0.047381,-0.003047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hot Potato,-0.058953,-0.032931,-0.127416,0.046263,-0.065068,-0.022222,0.100632,-0.101592,-0.031969,0.048770,...,-0.046479,-0.093698,0.006776,0.042431,0.090799,0.052930,-0.016827,0.006115,-0.118487,-0.011561
One Night of Love,-0.059397,-0.121556,-0.019065,-0.020089,-0.143418,0.031162,0.058256,0.016226,-0.019162,-0.144606,...,0.007160,-0.020355,0.011194,0.047112,0.085196,0.047241,-0.076121,-0.000444,0.044953,-0.091697
Lucky Numbers,-0.074399,0.016632,-0.004580,-0.062739,-0.085249,0.019829,0.132874,0.056214,0.041988,-0.068564,...,0.026873,-0.137031,-0.019027,-0.003781,-0.043318,-0.032821,-0.043339,-0.063091,-0.026474,0.057520


In [16]:
from sklearn.neighbors import NearestNeighbors

In [17]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(df_)

In [18]:
def find_similar_movies(df, nbrs, title):
    distances, indices = nbrs.kneighbors([df.loc[title]])
    #print(indices)#we print df data, no longer df_
    for index in indices[0][1:]:
        print('index', index)
        print(title, '->', df.iloc[index].name)

In [19]:
find_similar_movies(df_, nbrs, "Dungeons & Dragons: The Book of Vile Darkness")

index 1706
Dungeons & Dragons: The Book of Vile Darkness -> The Redeemer: Son of Satan!
index 1033
Dungeons & Dragons: The Book of Vile Darkness -> Mørke sjeler
index 252
Dungeons & Dragons: The Book of Vile Darkness -> The Eschatrilogy: Book of the Dead
index 4427
Dungeons & Dragons: The Book of Vile Darkness -> Teenage Caveman
index 1046
Dungeons & Dragons: The Book of Vile Darkness -> Berserker


In [20]:
def input_prompt(df, prompt):
    last_ind = len(df)
    encoded_prompt = model.encode(prompt)
    new_data = np.transpose(pd.DataFrame(list(zip(encoded_prompt))))
    df = pd.concat([df,new_data])
    nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(df)
    distances, indices = nbrs.kneighbors([df.iloc[last_ind, :]])
    #print(indices)#we print df data, no longer df_
    for index in indices[0][1:]:
        print(df.iloc[index].name)

In [21]:
prompt="drug addict in america looking for work"
input_prompt(df_, prompt)

Cocaine Cowboys
Puppet on a Chain
Maria Full of Grace
The Organization
The Seven Five


In [22]:
def input_prompt_philipp(df, prompt):
    encoded_prompt = model.encode(prompt)
    new_data = pd.DataFrame(np.transpose(list(zip(encoded_prompt))))

    distances, indices = nbrs.kneighbors(new_data)

    list_of_titles = []
    for index in indices[0][0:]:
        list_of_titles.append(df.iloc[index].name)
    #return df[df.index.isin(list_of_titles)]
    return list_of_titles

prompt="drug addict in america looking for work"
result = input_prompt_philipp(df_, prompt)
result


['Cocaine Cowboys',
 'Puppet on a Chain',
 'Maria Full of Grace',
 'The Organization',
 'The Seven Five',
 'Charas: A Joint Effort']

In [23]:
df_500.shape

(4771, 6)

In [24]:
def mini_lm_encode(df: pd.DataFrame) -> pd.DataFrame:
    '''
    convert the plot_synopsis to a vector using the MiniLM model
    input: df: pd.DataFrame
    output: df_encoded: pd.DataFrame
    title becomes the index
    remaining columns dropped
    '''
    model = SentenceTransformer('all-MiniLM-L6-v2')
    df_encoded = df.copy()
    df_encoded['plot_synopsis'] = df_encoded['plot_synopsis'].apply(lambda x : model.encode(x))
    df_index = df_encoded.pop('title')
    df_encoded = df_encoded[['plot_synopsis']]
    df_encoded = pd.DataFrame(np.column_stack(list(zip(*df_encoded.values))))
    df_encoded.index = df_index
    return df_encoded

In [26]:

prompt = "drug addict in america looking for work"
prompt_df = pd.DataFrame({'title': ['prompt'], 'plot_synopsis': [prompt]})
prompt_df


,title,plot_synopsis
0,prompt,drug addict in america looking for work


In [27]:
mini_lm_encode(prompt_df)

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
title,,,,,,,,,,,,,,,,,,,,,
prompt,0.006858,-0.099995,-0.028351,0.036328,-0.023539,0.037387,0.060802,-0.02606,-0.066034,-0.066332,...,0.037231,-0.012793,0.034264,-0.020711,-0.066482,-0.036823,0.055675,-0.045704,0.012043,0.034241


In [32]:
test_df = mini_lm_encode(df_100)
test_df.index.name = None
test_df

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
Blue Like Jazz,0.096135,-0.020363,-0.062729,-0.010408,-0.015675,0.090753,0.095977,0.010494,0.097868,-0.086392,...,0.037165,-0.092996,-0.093270,0.080303,-0.057393,-0.083403,-0.011385,0.020992,-0.010889,-0.105495
Alice in Wonderland,0.051691,0.000177,-0.013128,0.013611,0.017848,0.022855,0.103709,-0.029421,-0.012518,0.034303,...,-0.035835,-0.029663,-0.006456,0.039253,0.024700,0.104760,0.067567,0.022312,-0.020455,-0.065566
Die Wilden Kerle 2,0.008699,0.010941,-0.015267,0.018879,0.117334,0.004730,0.001750,-0.052740,0.094592,0.014123,...,-0.006519,-0.055640,-0.052131,0.045269,0.062107,-0.029135,0.046276,0.007522,-0.039173,0.008730
Verdades Secretas,-0.048113,-0.005419,-0.062633,0.066073,-0.044058,0.017565,0.139030,0.032871,0.006883,-0.042341,...,-0.038109,-0.024061,0.011849,0.054127,-0.053796,0.041282,0.042872,-0.072180,-0.031452,0.010564
Thor & Loki: Blood Brothers,0.020749,-0.009804,-0.004819,0.044869,0.034313,0.021528,-0.024415,-0.003244,0.101175,-0.025896,...,-0.035233,-0.042553,-0.006789,-0.031789,-0.060375,0.030923,0.044663,-0.073926,0.015335,-0.052170
Mark of the Lash,-0.100185,0.052688,0.005159,0.058014,0.009166,0.116315,-0.033137,0.011330,0.065535,0.024112,...,0.023444,-0.068496,0.015806,0.034503,-0.106604,-0.008253,0.127584,0.047080,0.019248,-0.027932
Yateem,-0.131233,0.009757,-0.041496,-0.019799,0.021739,0.013487,0.067533,0.020627,0.041833,-0.007522,...,0.012343,-0.084414,-0.040847,-0.000967,-0.034794,0.029937,0.073678,0.021075,0.051425,0.013163


In [1]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Create some sample data
X = np.array([[0, 1], [1, 1], [2, 2]])

# Create a NearestNeighbors object with n_neighbors = 2
nbrs = NearestNeighbors(n_neighbors=2, algorithm='auto').fit(X)

# Find the k-neighbors of each point in X
distances, indices = nbrs.kneighbors(X)

print("Indices of nearest neighbors:")
print(indices)


Indices of nearest neighbors:
[[0 1]
 [1 0]
 [2 1]]
